# Creating a basic chat experience with kernel arguments

In this example, we show how you can build a simple chat bot by sending and updating the kernel arguments with your requests. 

We introduce the Kernel Arguments object which in this demo functions similarly as a key-value store that you can use when running the kernel.

The chat history is local (i.e. in your computer's RAM) and not persisted anywhere beyond the life of this Jupyter session.

In future examples, we will show how to persist the chat history on disk so that you can bring it into your applications.  

In this chat scenario, as the user talks back and forth with the bot, the chat context gets populated with the history of the conversation. During each new run of the kernel, the kernel arguments and chat history can provide the AI with its variables' content. 

In [1]:
!pip install semantic-kernel==0.9.2b1

In [5]:
from services import Service

# Select a service to use for this notebook (available services: OpenAI, AzureOpenAI, HuggingFace)
selectedService = Service.AzureOpenAI

In [6]:
import semantic_kernel as sk
import semantic_kernel.connectors.ai.open_ai as sk_oai
from semantic_kernel.prompt_template.input_variable import InputVariable
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.functions.kernel_arguments import KernelArguments

kernel = sk.Kernel()

service_id = None
if selectedService == Service.OpenAI:
    from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

    api_key, org_id = sk.openai_settings_from_dot_env()
    service_id = "oai_chat_gpt"
    kernel.add_service(
        OpenAIChatCompletion(service_id=service_id, ai_model_id="gpt-3.5-turbo-1106", api_key=api_key, org_id=org_id),
    )
elif selectedService == Service.AzureOpenAI:
    from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    service_id = "aoai_chat_completion"
    kernel.add_service(
        AzureChatCompletion(service_id=service_id, deployment_name=deployment, endpoint=endpoint, api_key=api_key),
    )

Let's define a prompt outlining a dialogue chat bot.

In [7]:
prompt = """
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$user_input}}
ChatBot: """

Register your semantic function

In [8]:
if selectedService == Service.OpenAI:
    execution_settings = sk_oai.OpenAIChatPromptExecutionSettings(
        service_id=service_id,
        ai_model_id="gpt-3.5-turbo-1106",
        max_tokens=2000,
        temperature=0.7,
    )
elif selectedService == Service.AzureOpenAI:
    execution_settings = sk_oai.OpenAIChatPromptExecutionSettings(
        service_id=service_id,
        ai_model_id=deployment,
        max_tokens=2000,
        temperature=0.7,
    )

prompt_template_config = sk.PromptTemplateConfig(
    template=prompt,
    name="chat",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="input", description="The user input", is_required=True),
        InputVariable(name="history", description="The conversation history", is_required=True),
    ],
    execution_settings=execution_settings,
)

chat_function = kernel.create_function_from_prompt(
    function_name="chat",
    plugin_name="chatPlugin",
    prompt_template_config=prompt_template_config,
)

In [9]:
chat_history = ChatHistory()
chat_history.add_system_message("You are a helpful chatbot who is good about giving book recommendations.")

Initialize the Kernel Arguments

In [10]:
arguments = KernelArguments(user_input="Hi, I'm looking for book suggestions", history=chat_history)

Chat with the Bot

In [11]:
response = await kernel.invoke(chat_function, arguments)
print(response)

Hello! I'd be happy to help you with book suggestions. What genre or type of book are you interested in?


Update the history with the output

In [12]:
chat_history.add_assistant_message(str(response))

Keep Chatting!

In [13]:
async def chat(input_text: str) -> None:
    # Save new message in the context variables
    print(f"User: {input_text}")
    chat_history.add_user_message(input_text)

    # Process the user message and get an answer
    answer = await kernel.invoke(chat_function, KernelArguments(user_input=input_text, history=chat_history))

    # Show the response
    print(f"ChatBot: {answer}")

    chat_history.add_assistant_message(str(answer))

In [14]:
await chat("I love history and philosophy, I'd like to learn something new about Greece, any suggestion?")

User: I love history and philosophy, I'd like to learn something new about Greece, any suggestion?
ChatBot: Certainly! If you're interested in Greek history and philosophy, a great book to consider is "The Greeks" by H.D.F. Kitto. It provides a comprehensive overview of ancient Greek civilization, covering various aspects such as politics, art, literature, and philosophy. It's a great way to delve into the rich cultural heritage of Greece and gain a deeper understanding of its influence on Western civilization.


In [15]:
await chat("that sounds interesting, what is it about?")

User: that sounds interesting, what is it about?
ChatBot: "The Greeks" by H.D.F. Kitto is a book that explores the history and culture of ancient Greece. It delves into various aspects of Greek civilization, such as their political systems, art, literature, and philosophy. The book provides insights into the development of democracy, the influence of Greek mythology, the works of famous philosophers like Socrates, Plato, and Aristotle, and the impact of Greek drama and poetry. It offers a comprehensive and engaging overview of ancient Greece, making it a great option for anyone interested in learning more about this fascinating period in history.


In [16]:
await chat("if I read that book, what exactly will I learn about Greek history?")

User: if I read that book, what exactly will I learn about Greek history?
ChatBot: If you read "The Greeks" by H.D.F. Kitto, you will learn about various aspects of Greek history. Some of the key topics covered in the book include:

1. The rise and fall of the city-states: You'll learn about the development of city-states like Athens and Sparta, their political systems, and their conflicts with each other.

2. Democracy: The book explores the birth of democracy in ancient Athens and its impact on Greek society and politics.

3. Greek mythology and religion: You'll gain insights into Greek mythology, the stories of gods and goddesses, and their significance in Greek culture.

4. Philosophy: The book delves into the works of famous Greek philosophers like Socrates, Plato, and Aristotle, exploring their ideas and their influence on Western thought.

5. Art and architecture: You'll learn about Greek art forms, including sculpture and architecture, and their contributions to the field.

6. 

In [17]:
await chat("could you list some more books I could read about this topic?")

User: could you list some more books I could read about this topic?
ChatBot: Certainly! Here are a few more book recommendations on Greek history and philosophy:

1. "The History of the Peloponnesian War" by Thucydides - This classic work is a detailed account of the war between Athens and Sparta, providing valuable insights into ancient Greek politics and society.

2. "The Republic" by Plato - In this philosophical dialogue, Plato explores the concept of justice and presents his ideal state, offering profound insights into political philosophy.

3. "The Iliad" by Homer - This epic poem is one of the oldest works of Western literature, recounting the events of the Trojan War and showcasing themes of heroism, honor, and fate.

4. "The Birth of Tragedy" by Friedrich Nietzsche - Nietzsche examines the origins and significance of Greek tragedy, exploring the interplay between the Apollonian and Dionysian forces in art.

5. "The Trial and Death of Socrates" by Plato - This dialogue depicts 

After chatting for a while, we have built a growing history, which we are attaching to each prompt and which contains the full conversation. Let's take a look!

In [18]:
print(chat_history)

<message role="system" metadata="{}">You are a helpful chatbot who is good about giving book recommendations.</message>
<message role="assistant" metadata="{}">Hello! I'd be happy to help you with book suggestions. What genre or type of book are you interested in?</message>
<message role="user" metadata="{}">I love history and philosophy, I'd like to learn something new about Greece, any suggestion?</message>
<message role="assistant" metadata="{}">Certainly! If you're interested in Greek history and philosophy, a great book to consider is "The Greeks" by H.D.F. Kitto. It provides a comprehensive overview of ancient Greek civilization, covering various aspects such as politics, art, literature, and philosophy. It's a great way to delve into the rich cultural heritage of Greece and gain a deeper understanding of its influence on Western civilization.</message>
<message role="user" metadata="{}">that sounds interesting, what is it about?</message>
<message role="assistant" metadata="{}">